In [4]:
import requests
import pandas as pd

# Função para baixar dados da API do BCB
def baixar_serie_bcb(codigo_serie, nome_serie):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
        df['valor'] = pd.to_numeric(df['valor'])
        df.rename(columns={'valor': nome_serie}, inplace=True)
        return df
    else:
        print(f"Erro ao baixar a série {nome_serie}: {response.status_code}")
        return None

In [6]:


# Baixando as séries
df_ipca = baixar_serie_bcb(433, 'IPCA')
df_selic = baixar_serie_bcb(432, 'Selic')
df_cambio = baixar_serie_bcb(10813, 'Câmbio USD/BRL')
display (df_ipca)



Erro ao baixar a série Selic: 406
Erro ao baixar a série Câmbio USD/BRL: 406


,data,IPCA
0,1980-02-01,4.62
1,1980-03-01,6.04
2,1980-04-01,5.29
3,1980-05-01,5.70
4,1980-06-01,5.31
...,...,...
536,2024-10-01,0.56
537,2024-11-01,0.39
538,2024-12-01,0.52
539,2025-01-01,0.16


In [8]:
import requests

# Exemplo de URL para série Selic
url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json"

response = requests.get(url)

if response.status_code == 200:
    print("Dados da série Selic:", response.json())
else:
    print(f"Erro {response.status_code} ao tentar acessar a série Selic.")

Erro 406 ao tentar acessar a série Selic.


In [ ]:
# Combinando as séries em um único DataFrame
if df_ipca is not None and df_selic is not None and df_cambio is not None:
    df_final = pd.merge(df_ipca, df_selic, on='data', how='outer')
    df_final = pd.merge(df_final, df_cambio, on='data', how='outer')
    
    # Ordenando por data
    df_final = df_final.sort_values(by='data',ascending=False )
    
    # Exibindo as primeiras linhas
    display(df_final.head())
    
    # Salvando em um arquivo CSV
    df_final.to_csv('previsoes_focus.csv', index=False)
    print("Dados salvos em 'previsoes_focus.csv'")
else:
    print("Erro ao baixar uma ou mais séries.")

In [10]:
import requests
import pandas as pd

# Função para baixar dados da API do BCB
def baixar_serie_bcb(codigo_serie, nome_serie):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
        df['valor'] = pd.to_numeric(df['valor'])
        df.rename(columns={'valor': nome_serie}, inplace=True)
        return df
    else:
        print(f"Erro ao baixar a série {nome_serie} (Código {codigo_serie}): {response.status_code}")
        return None

# Baixando as séries de previsão para 2026, 2027 e 2028
# Inflação (IPCA)
df_ipca_2026 = baixar_serie_bcb(11778, 'IPCA 2026')
df_ipca_2027 = baixar_serie_bcb(11779, 'IPCA 2027')
df_ipca_2028 = baixar_serie_bcb(11780, 'IPCA 2028')

# Taxa Selic
df_selic_2026 = baixar_serie_bcb(11781, 'Selic 2026')
df_selic_2027 = baixar_serie_bcb(11782, 'Selic 2027')
df_selic_2028 = baixar_serie_bcb(11783, 'Selic 2028')

# Câmbio (USD/BRL)
df_cambio_2026 = baixar_serie_bcb(11784, 'Câmbio 2026')
df_cambio_2027 = baixar_serie_bcb(11785, 'Câmbio 2027')
df_cambio_2028 = baixar_serie_bcb(11786, 'Câmbio 2028')

# Verificando se todas as séries foram baixadas corretamente
dataframes = {
    'IPCA 2026': df_ipca_2026,
    'IPCA 2027': df_ipca_2027,
    'IPCA 2028': df_ipca_2028,
    'Selic 2026': df_selic_2026,
    'Selic 2027': df_selic_2027,
    'Selic 2028': df_selic_2028,
    'Câmbio 2026': df_cambio_2026,
    'Câmbio 2027': df_cambio_2027,
    'Câmbio 2028': df_cambio_2028
}

# Filtrando apenas os DataFrames que não são None
dataframes_validos = {nome: df for nome, df in dataframes.items() if df is not None}

if not dataframes_validos:
    print("Erro: Nenhuma série foi baixada corretamente.")
else:
    # Combinando os DataFrames válidos
    df_final = None
    for nome, df in dataframes_validos.items():
        if df_final is None:
            df_final = df
        else:
            df_final = pd.merge(df_final, df, on='data', how='outer')
    
    # Ordenando por data
    df_final = df_final.sort_values(by='data')
    
    # Exibindo as primeiras linhas
    print(df_final.head())
    
    # Salvando em um arquivo CSV
    df_final.to_csv('previsoes_focus_2026_2028.csv', index=False)
    print("Dados salvos em 'previsoes_focus_2026_2028.csv'")

Erro ao baixar a série IPCA 2026 (Código 11778): 400
Erro ao baixar a série IPCA 2027 (Código 11779): 400
Erro ao baixar a série IPCA 2028 (Código 11780): 400
Erro ao baixar a série Selic 2026 (Código 11781): 400
Erro ao baixar a série Selic 2027 (Código 11782): 400
Erro ao baixar a série Selic 2028 (Código 11783): 400
Erro ao baixar a série Câmbio 2026 (Código 11784): 400
Erro ao baixar a série Câmbio 2027 (Código 11785): 400
Erro ao baixar a série Câmbio 2028 (Código 11786): 400
Erro: Nenhuma série foi baixada corretamente.
